In [2]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [65]:
query2=f"""
SELECT
A.*,
B.RoundNumber
FROM `perceptive-ivy-290216.f1_api.results_race` A
INNER JOIN
`perceptive-ivy-290216.f1_api.schedules_update` B
ON A.GP=B.EventName AND A.Year=B.Year
WHERE
A.Year=2024
AND B.Year=2024
-- AND
--  A.GP='Qatar Grand Prix'
"""

track2=pd.read_gbq(query2,project_id,dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3930/1015015979.py:16: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [66]:
query3=f"""
SELECT
A.*,
B.RoundNumber
FROM `perceptive-ivy-290216.f1_api.results_sprint` A
INNER JOIN
`perceptive-ivy-290216.f1_api.schedules_update` B
ON A.GP=B.EventName AND A.Year=B.Year
WHERE
A.Year=2024
-- AND
-- A.GP='Qatar Grand Prix'
"""
track3=pd.read_gbq(query3,project_id,dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3930/3282991377.py:14: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [67]:
#Points

In [68]:
points_table=track2[["RoundNumber","GP","Abbreviation","Points"]]
points_table["Points"]=points_table["Points"].astype(int)
points_table=points_table.sort_values(by=["RoundNumber","Abbreviation"])
points_table.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3930/3640686471.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,RoundNumber,GP,Abbreviation,Points
91,1,Bahrain Grand Prix,ALB,0
100,1,Bahrain Grand Prix,ALO,2
111,1,Bahrain Grand Prix,BOT,0
60,1,Bahrain Grand Prix,GAS,0
79,1,Bahrain Grand Prix,HAM,6


In [69]:
points_table_sprint=track3[["RoundNumber","GP","Abbreviation","Points"]]
points_table_sprint["Points"]=points_table_sprint["Points"].astype(int)
points_table_sprint=points_table_sprint.sort_values(by=["RoundNumber","Abbreviation"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3930/482007640.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [70]:
points_table=pd.concat([points_table,points_table_sprint])
points_table.head()

,RoundNumber,GP,Abbreviation,Points
91,1,Bahrain Grand Prix,ALB,0
100,1,Bahrain Grand Prix,ALO,2
111,1,Bahrain Grand Prix,BOT,0
60,1,Bahrain Grand Prix,GAS,0
79,1,Bahrain Grand Prix,HAM,6


In [54]:
#Get all drivers GP counts to fill in missing data and remove drivers who drove sparingly. The Other Option is to add the GP Data for sparing drivers to keep the dataset uniform
points_table.groupby(by=["Abbreviation"])["GP"].count()

Abbreviation
ALB    14
ALO    14
BEA     1
BOT    14
GAS    14
HAM    14
HUL    14
LEC    14
MAG    14
NOR    14
OCO    14
PER    14
PIA    14
RIC    14
RUS    14
SAI    13
SAR    13
STR    14
TSU    14
VER    14
ZHO    14
Name: GP, dtype: int64

In [71]:
#Find out which GPs were missed by drivers and then use the next block to fill in the details missing
# points_table[points_table["RoundNumber"].isin([16])].sort_values(by=["RoundNumber","DriverId"])
points_table[points_table["Abbreviation"]=="SAI"]

,RoundNumber,GP,Abbreviation,Points
44,1,Bahrain Grand Prix,SAI,15
46,3,Australian Grand Prix,SAI,25
45,4,Japanese Grand Prix,SAI,15
26,5,Chinese Grand Prix,SAI,10
174,6,Miami Grand Prix,SAI,10
131,7,Emilia Romagna Grand Prix,SAI,10
172,8,Monaco Grand Prix,SAI,15
171,9,Canadian Grand Prix,SAI,0
173,10,Spanish Grand Prix,SAI,8
4,11,Austrian Grand Prix,SAI,15


In [72]:
points_table=points_table[(points_table["Abbreviation"]!="HUL")&(points_table["Abbreviation"]!="DEV")]

In [73]:
#Adding Albon's Italian GP Data which is missing. This messes up the line chart, so to avoid that and uniformity in data adding this row here.
points_table.loc[-1] = [2,"Saudi Arabian Grand Prix", "SAI", 0] # adding a row

In [74]:
#Combine Race and Sprint in 1 row
points_table=pd.DataFrame(points_table.groupby(by=["RoundNumber","GP","Abbreviation"])["Points"].sum()).reset_index()
points_table=points_table.sort_values(by=["Abbreviation","RoundNumber"])
points_table["cumsum"]=points_table.groupby(by=["Abbreviation"])["Points"].cumsum()

#Create a new table that has totals by driver id which will be used to sort the final chart
totals=pd.DataFrame(points_table.groupby(by=["Abbreviation"])["Points"].sum())
totals.columns=["sum"]

#Merge both tables
points_table=points_table.merge(totals, left_on='Abbreviation', right_on='Abbreviation')

points_table=points_table.sort_values(by=["sum","Abbreviation","RoundNumber"], ascending=[False, True, True])
points_table.head()

,RoundNumber,GP,Abbreviation,Points,cumsum,sum
187,1,Bahrain Grand Prix,VER,26,26,237
188,2,Saudi Arabian Grand Prix,VER,25,51,237
189,3,Australian Grand Prix,VER,0,51,237
190,4,Japanese Grand Prix,VER,26,77,237
191,5,Chinese Grand Prix,VER,33,110,237


In [75]:
fig=px.line(
    points_table,
    y="cumsum",
    x='GP',
    # text='cumsum',
    # text='LapTimeDelta2',
    color='Abbreviation',
    title="<b>Driver's Championship Points</b>",
    height=1200, width=1600    ,
    template="plotly_dark",

           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>POINTS</b>",
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig.show()

In [79]:
fig=px.scatter(
    points_table,
    y="Abbreviation",
    x='cumsum',
    color='Abbreviation',
    # text="cumsum",
    title="<b>Driver's Championship Points</b>",
    height=1000, width=1200,
    animation_frame="RoundNumber",
    animation_group="Abbreviation",
    range_x=[0,500],
    # range_y=[0,500],
    size='cumsum',
    # text='cumsum',

           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)
fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    xaxis_title="<b>POINTS</b>",
    yaxis_title="<b>DRIVER</b>",
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig.show()

In [83]:
points_table['cumsum']=points_table['cumsum'].astype(int)

In [84]:
fig=px.bar(
    points_table,
    y="cumsum",
    x='Abbreviation',
    color='Abbreviation',
    text='cumsum',
    title="<b>Driver's Championship Points</b>",
    height=1000, width=1200,
    animation_frame="RoundNumber",
    animation_group="Abbreviation",
    # range_x=[0,500],
    range_y=[0,500],
    template="plotly_dark",
    # size='cumsum',
    # text='cumsum',

           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)
fig.update_traces(textposition='outside')

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    xaxis_title="<b>DRIVER</b>",
    yaxis_title="<b>POINTS</b>",
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig.show()